<a href="https://colab.research.google.com/github/darshana-22/Music-Genre-Classification/blob/main/Implementing%20the%20neural%20network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import scipy
import sys
import os
import pickle
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Data.zip"

In [ ]:
DATASET_PATH = "/content/genres_original"
JSON_PATH = "data.json"
SAMPLE_RATE = 22050
DURATION = 30 # in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

In [ ]:
def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
  # dictionary to store data
  data={
      "mapping": [],
      "mfcc":[],
      "labels": []
  }
  num_samples_per_segment = int (SAMPLES_PER_TRACK / num_segments)
  expected_num_mfcc_vectors_per_segment = math.ceil(num_samples_per_segment/ hop_length)

  for i, (dirpath, dirnames, filenames) in enumerate (os.walk(dataset_path)):
    if dirpath is not dataset_path:
      # save the semantic label
      dirpath_components = dirpath.split("/")
      semantic_label = dirpath_components[-1]
      data["mapping"].append(semantic_label)
      print("\nprocessing {}".format(semantic_label))

      for f in filenames:
        #loading audio file
        file_path = os.path.join(dirpath, f)
        signal, sr = librosa.load(file_path, sr= SAMPLE_RATE)

        # process segments extracting mfcc and storing data
        for s in range(num_segments):
          start_sample = num_samples_per_segment * s
          finish_sample = start_sample + num_samples_per_segment


          mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                      sr = sr,
                                      n_fft = n_fft,
                                      n_mfcc = n_mfcc ,
                                      hop_length = hop_length)
          
          
          mfcc = mfcc.T
          if len(mfcc) == expected_num_mfcc_vectors_per_segment:
            data["mfcc"].append(mfcc.tolist())
            data["labels"].append(i-1)
            print("{}, segment:{}".format(file_path, s+1))

  with open(json_path, "w") as fp:
     json.dump(data, fp, indent=4)   






In [ ]:
save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)

In [28]:
DATASET_PATH = "/content/data.json"

In [36]:
def load_data(dataset_path):
  with open(dataset_path, "r") as fp:
    data= json.load(fp)

    #convert lists into numpy arrays
    inputs = np.array(data["mfcc"])
    targets = np.array(data["labels"])
    print("Data successfully loaded!!")

    return inputs, targets

In [37]:
inputs, targets = load_data(DATASET_PATH)

Data successfully loaded!!


In [ ]:
#split the data into train and test set
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs,targets, test_size= 0.3)

#build the model
model = keras.Sequential([
                          #input layer
                          keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),

                          #1st hidden layer
                          keras.layers.Dense(512, activation="relu"),

                          #2nd hidden layer
                          keras.layers.Dense(256, activation="relu"),

                          #3rd hidden layer
                          keras.layers.Dense(64, activation="relu"),

                          #output layer
                          keras.layers.Dense(11, activation="softmax")
])

optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss = "sparse_categorical_crossentropy",
              metrics = ["accuracy"])
model.summary()








In [ ]:
#train the network
model.fit(inputs_train, targets_train, 
          validation_data=(inputs_test, targets_test), 
          batch_size=32,
          epochs=50)
